## Importing necessary libraries

In [181]:
import pandas as pd
import re
import inflect
import numpy as np
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [210]:
# Read data into pandas DataFrames
reviews = pd.read_csv('./train_labelled.csv', encoding = 'unicode_escape')
# reviews = pd.read_csv('./train_cleaned.csv')

In [211]:
reviews.isnull().any()

Restaurant    False
Review        False
Label         False
Stars         False
Date          False
dtype: bool

In [212]:
reviews.shape

(3463, 5)

In [179]:
reviews.head()

,Restaurant,Review,Label,Stars,Date
0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00
1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00
2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00
3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00
4,Brawn & Brains Coffee,I don't get why it can't just be full service....,1,3,2019-12-17T00:00:00


## Data Preprocessing

In [ ]:
# Build vocabulary of words and their respective counts
corpus = []

for line in reviews['Review']:
    line = line.replace('\n', ' ').replace('\t', ' ').lower() # remove next line, tabs and convert to lower casing
    line = re.sub('[^a-z]', ' ', line) # remove punctuations and numbers
    tokens = line.split(' ')
    tokens = [token for token in tokens if len(token) > 0]
    corpus.extend(tokens)

corpus = Counter(corpus)
print('Number of words before cleaning: ', len(corpus))
# 5962 without numbers

In [232]:
# Cleaning the Data
def clean(data):
    
    # Removing leading and trailing white spaces
    data = str(data).strip()
    
    # Converting all text to lower case
    data = data.lower() 
    
    # Adding space for punctuation
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    translator.sub(' ', data)

    # Removing punctuation
    data = re.sub('[^A-Za-z0-9]+', ' ', data)
    
    # Keeping only english words
    data = re.sub(r'[^\x00-\x7F]+', '', data)
    
    # Keeping printable
    data = re.sub(f'[^{re.escape(string.printable)}]', '', data)

    # Tokenizing data
    tokens = word_tokenize(data)

    # Correcting spelling mistakes
    correct_text = spell_corrector(tokens, corpus)

    # Removing Stopwords
#     stop_words = set(stopwords.words('english'))

    # Converting all text to base form
#     lemmatizer = WordNetLemmatizer()
#     tokens_lemmatized = [lemmatizer.lemmatize(token) for token in tokens]

    # Removing Punctuation
#     translator = str.maketrans('', '', string.punctuation)
#     data = data.translate(translator)

    # Removing all the numbers
#     word = data.split(' ')
#     #p = inflect.engine()
#     #p.number_to_words(word)
#     word = ' '.join([i for i in word if not i.isdigit()])
#     word = re.sub(r'\d+','' , word)

    # Converting all numbers to words
#    word = data.split(' ')
#     p = inflect.engine()
#     word = ' '.join([i for i in word if not i.isdigit()])
#     word = re.sub(r'\d+', p.number_to_words(word) , word)  

    # Removing all the odd characters with the aprostroe etc
#     word = re.sub(r'([^\s\w]|_)+', "", word)
#     word = re.sub(r'ý+', '', word)

#     word = ' '.join([i for i in tokens_lemmatized])

    return correct_text

# clean text data
reviews["Review_clean"] = reviews["Review"].apply(lambda x: clean(x))
reviews

,Restaurant,Review,Label,Stars,Date,Review_clean
0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00,fresh ingredient friendly peep much cheaper sa...
1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00,small selection italian wine glass beer well g...
2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00,plus point price remains 4 per bowl
3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00,ordered thought wa signature omelette name
4,Brawn & Brains Coffee,I don't get why it can't just be full service....,1,3,2019-12-17T00:00:00,get full service menu wa decent price pretty e...
...,...,...,...,...,...,...
3458,Momma Kong's Crab Shack,They had a not so great experience with sotong...,5,3,2015-05-23T00:00:00,great experience sotong last time decided give...
3459,Spago By Wolfgang Puck,It'll take you up to the 50 something floor wh...,5,4,2018-01-10T00:00:00,take 50 something floor need transfer another ...
3460,Swee Choon Tim Sum,Tips: Bring cash cause they don't accept credi...,5,5,2019-10-19T00:00:00,tip bring cash cause accept credit card
3461,Sakae Teppanyaki,The only reason why I visited this store in th...,5,5,2012-08-17T00:00:00,reason visited store first place wa voucher


In [ ]:
# Build vocabulary of cleaned text and their respective counts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews.Review_clean)
print("Number of words after cleaning: ", len(tokenizer.word_index))
# 5238 lemmatizing -> correct spelling error (charlotte code)
# 3728 lemmatizing -> correct spelling error (my code)
# 6096 correct spelling error with numbers without lemmatizing (charlotte code)
# 5942 correct spelling error without numbers without lemmatizing (charlotte code)

In [214]:
# To get unique restaurant names
restaurant_names = []
for rest in reviews["Restaurant"]: # each restaurant name
    if rest not in restaurant_names: 
        restaurant_names.append(rest)

print(len(restaurant_names)) # number of restaurants

595


In [215]:
# To get restaurant name and respective reviews (separately including date)
reviewsByRestaurant = dict.fromkeys(restaurant_names, "")
for rest in restaurant_names: 
    filtered= []
    reviewsByRestaurant[rest] = [] # to contain a list of sentences for that restaurant    
    filtered = reviews[reviews['Restaurant'] == rest]
    for i in range(len(filtered)): 
        review_info = {} # to contain key value pairs of label: sentence, date: date
        review_info['Review'] = filtered.iloc[i]['Review_clean']
        review_info['Date'] = pd.to_datetime(filtered.iloc[i]['Date'])
        review_info['Label'] = filtered.iloc[i]['Label']
        review_info['Stars'] = filtered.iloc[i]['Stars']
        reviewsByRestaurant[rest].append(review_info)

print(reviewsByRestaurant)

{'Happy Tummy': [{'Review': 'fresh ingredient friendly peep much cheaper salad stop single get base romaine 5 topping 6 80', 'Date': Timestamp('2016-04-06 00:00:00'), 'Label': 1, 'Stars': 5}, {'Review': 'ideal location right beneath classroom making definite choice lunch dinner class', 'Date': Timestamp('2015-11-17 00:00:00'), 'Label': 4, 'Stars': 5}, {'Review': 'definitely good option traditional breakfast guess try salad people raving', 'Date': Timestamp('2017-03-03 00:00:00'), 'Label': 5, 'Stars': 4}], 'Cibo Italiano': [{'Review': 'small selection italian wine glass beer well good wine list', 'Date': Timestamp('2015-12-24 00:00:00'), 'Label': 1, 'Stars': 4}, {'Review': 'generous clam', 'Date': Timestamp('2016-11-28 00:00:00'), 'Label': 1, 'Stars': 5}, {'Review': 'cultural relevant singaporean cuisine sweet old conservation bldg', 'Date': Timestamp('2018-08-19 00:00:00'), 'Label': 4, 'Stars': 5}], 'Yan kee Noodle House': [{'Review': 'plus point price remains 4 per bowl', 'Date': Time

In [216]:
# To get df of restaurant and labels with date
rows = []

for rest in restaurant_names:
    for review_info in reviewsByRestaurant[rest]:
        review_info['Restaurant'] = rest
        rows.append(review_info)

reviews_by_restaurant = pd.DataFrame(rows)
reviews_by_restaurant = reviews_by_restaurant[['Restaurant', 'Review', 'Date', 'Label', 'Stars']]

reviews_by_restaurant.head()

,Restaurant,Review,Date,Label,Stars
0,Happy Tummy,fresh ingredient friendly peep much cheaper sa...,2016-04-06,1,5
1,Happy Tummy,ideal location right beneath classroom making ...,2015-11-17,4,5
2,Happy Tummy,definitely good option traditional breakfast g...,2017-03-03,5,4
3,Cibo Italiano,small selection italian wine glass beer well g...,2015-12-24,1,4
4,Cibo Italiano,generous clam,2016-11-28,1,5


In [127]:
# To get restaurant name and respective reviews (concatenated)
reviewsByRestaurant = dict.fromkeys(restaurant_names, "")
for rest in restaurant_names: 
    filtered= []
    reviewsByRestaurant[rest] = {} # create a dictionary for sentences (key = label)
    labels = []
    for label in reviews['Label'][reviews['Restaurant'] == rest]:
        if label not in labels:
            labels.append(label)
#     for cat in labels: 
    for i in range(1, 6):
        reviewsByRestaurant[rest][i] = '' # to compile sentences for a label in a restaurant
        for j in labels:
            if i == j:
                filtered= reviews['Review_clean'][(reviews['Restaurant'] == rest) & (reviews['Label'] == j)] # obtain reviews for a label in a restaurant
                for rev in filtered:
                    reviewsByRestaurant[rest][j] += rev

reviewsByRestaurant

{'Happy Tummy': {1: 'fresh ingredients friendly peeps and so much cheaper than salad stop a single gets you a base of romaine and 5 toppings for 6 80',
  2: '',
  3: '',
  4: 'ideal location right beneath my classroom making it a definite choice for me for lunch or dinner before and after class',
  5: 'definitely a good option for a traditional breakfast and i guess i should try the salads people are raving about'},
 'Cibo Italiano': {1: 'a small selection of italian wines by the glass and beers as well as a good wine listgenerous with the clams',
  2: '',
  3: '',
  4: 'cultural relevant singaporean cuisine in very sweet old conservation bldg',
  5: ''},
 'Yan kee Noodle House': {1: 'the plus point is that the price remains the same at 4 per bowl',
  2: '',
  3: '',
  4: '',
  5: ''},
 'Clinton Street Baking Company & Restaurant': {1: 'same for more i ordered what i thought was their signature omelette because it had their name on itit looked awful and for 18 i was not happyall were e

In [128]:
# To get df of restaurant and labels with date
reviews_by_restaurant = pd.DataFrame.from_dict(reviewsByRestaurant)
reviews_by_restaurant = reviews_by_restaurant.transpose()
reviews_by_restaurant = reviews_by_restaurant.reset_index()
# reviews_by_restaurant = reviews_by_restaurant.rename(columns={"index": "Restaurant"})
reviews_by_restaurant.columns =['Restaurant', 'Value', 'Taste', 'Service', 'Ambience', 'Others'] 

reviews_by_restaurant

,Restaurant,Value,Taste,Service,Ambience,Others
0,Happy Tummy,fresh ingredients friendly peeps and so much c...,,,ideal location right beneath my classroom maki...,definitely a good option for a traditional bre...
1,Cibo Italiano,a small selection of italian wines by the glas...,,,cultural relevant singaporean cuisine in very ...,
2,Yan kee Noodle House,the plus point is that the price remains the s...,,,,
3,Clinton Street Baking Company & Restaurant,same for more i ordered what i thought was the...,each component from the just cooked spinach to...,service was prompt but i guess a true test of ...,4 stars for the ambience,place is a staplewould go back once in a while...
4,Brawn & Brains Coffee,i don t get why it can t just be full service ...,but the coffee and food is great,the service was welcoming and friendlyfree wif...,this place is pretty cute and definitely remin...,situated at the front of the old badminton com...
...,...,...,...,...,...,...
585,Brazil Churrasco,,,and attentive,,
586,Marrakesh Moroccan Lounge,,,,i generally dislike clarke quay because it s r...,
587,Fotia,,,,i ve been to their earlier location which was ...,
588,Idealite,,,,the inside was nice and cool,


## Word Embeddings

In [111]:
from gensim.models import Word2Vec

all_sentences = [nltk.sent_tokenize(txt) for txt in x_train] # splits into sentences
all_sentences

[['i may be demanding on this but i did not foresee getting my hand dirty trying to peel the meat off the crab in a fancy italian restaurant'],
 ['can t wait to come back for more'],
 ['it a spot i like to take people who are visiting because you wont find it anywhere else'],
 ['not expensive either'],
 ['at 9 i found the price steep initially but the portion wa huge'],
 ['visited in a large group of 9 people'],
 ['it wa juicy well seasoned and incredibly rich'],
 ['i had the ebi tempura with pork broth udon and it seriously reminded me of the same meal at a fast food restaurant in the train station in hakata fukuoka'],
 ['the pork cheek are so soft they melt in your mouth'],
 ['great service and friendly staff'],
 ['while there re so many new thai place that sprout up around the city i love being able to head to a reliable place like this know i ll have a tasty meal always'],
 ['the choice are few but high quality'],
 ['my colleague had boeuf bourguignon which looked a bit predictable

In [28]:
all_words = []

for sentence in all_sentences: 
    for text in sentence:
        
all_words = [nltk.word_tokenize(text) for text in sentence for sentence in all_sentences]
all_words

TypeError: expected string or bytes-like object

In [ ]:
word2vec = Word2Vec(text, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [ ]:
vocabulary = word2vec.wv
print(vocabulary)

## Correct spelling mistakes

In [ ]:
# for each word
int maxEditDistanceLookup = 1; //max edit distance per lookup (maxEditDistanceLookup<=maxEditDistanceDictionary)
var suggestionVerbosity = SymSpell.Verbosity.Closest; //Top, Closest, All
var suggestions = symSpell.Lookup(token, suggestionVerbosity, maxEditDistanceLookup);

## POS Tagging

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

#example text text = 'What can I say about this place. The staff of these restaurants is nice and the eggplant is not bad'

class Splitter(object):
    """
    split the document into sentences and tokenize each sentence
    """
    def __init__(self):
        self.splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):
        # split into single sentence
        sentences = self.splitter.tokenize(text)
        # tokenization in each sentences
        tokens = [self.tokenizer.tokenize(sent) for sent in sentences]
        return tokens


class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(word, lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]
        return pos_tokens[0]

lemmatizer = WordNetLemmatizer()
splitter = Splitter()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

#step 1 split document into sentence followed by tokenization
# reviews["Review_tokenized"] = reviews["Review_clean"].apply(lambda x: splitter.split(x))

#step 2 lemmatization using pos tagger 
reviews["Review_pos_tagged"] = reviews["Review_clean"].apply(lambda x: lemmatization_using_pos_tagger.pos_tag(x))

In [ ]:
def get_pos_tags(column):
    for lst in column:
        for 

In [ ]:
# Creating a WordCloud to analyse the data
long_string = ','.join(list(x[1] in reviews['content_pos_tagged'].iloc[0] if x[2][0] == 'NN')
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=100, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

## Classification

In [218]:
X = reviews.Review_clean # feature matrix
y = reviews.Label # response vector

X.head()

0    Fresh ingredient friendly peep much cheaper Sa...
1    A small selection Italian wine glass beer well...
2              The plus point price remains 4 per bowl
3    Same I ordered I thought wa signature omelette...
4    I get full service The menu wa decent price pr...
Name: Review_clean, dtype: object

In [219]:
print(X[1])

A small selection Italian wine glass beer well good wine list


In [220]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 1)
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)

In [55]:
# Define a function that accepts a vectorizer, a model and calculates the accuracy
def tokenize_predict(vect, model):
    
    # create document-term matrices using the vectorizer
    x_train_dtm = vect.fit_transform(x_train)
    x_test_dtm = vect.transform(x_test)
    
    # print the number of features that were generated
    print('Features: ', x_train_dtm.shape[1])
    
    model.fit(x_train_dtm, y_train)
    
    trainpred = model.predict(x_train_dtm) # for train data
    testpred = model.predict(x_test_dtm) # for test data
    
    # print the accuracy of predictions
    print('Train Accuracy: ', metrics.accuracy_score(y_train, trainpred))
    print('Test Accuracy: ', metrics.accuracy_score(y_test, testpred))
    print('Train AUROC: ', metrics.roc_auc_score(y_train, trainpred))
    print('Test AUROC: ', metrics.roc_auc_score(y_test, testpred))

In [ ]:
# Lemmatize words
class LemmaTokenizer:
     def __init__(self):
         self.wnl = WordNetLemmatizer()

     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
# Import and instantiate Count Vectorizer
vect = CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                , encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=LemmaTokenizer(),
                vocabulary=None)

## Logreg

In [ ]:
logreg = LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
# (setting multi_class=”ovr”)
# (setting multi_class="multinomial")

## RNN

In [10]:
# Import necessary libraries
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential

Using TensorFlow backend.


In [15]:
# Lemmatize training data for rnn
# lemmatized = []

# lemmatizer = WordNetLemmatizer()

# for sentence in x_train:
#     word_list = nltk.word_tokenize(sentence)
#     lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
#     lemmatized.append(lemmatized_output)
    
# train_data = lemmatized

In [221]:
# Build input sequence for running the model
max_len = 150
num_words = 1000
tok = Tokenizer(num_words= num_words)
tok.fit_on_texts(x_train)
word_idx = tok.word_index

In [222]:
# Padding sequences
sequences = tok.texts_to_sequences(x_train)
train_matrix = sequence.pad_sequences(sequences,maxlen= max_len)
# test_sequences = tok.texts_to_sequences(x_test)
# test_matrix = sequence.pad_sequences(test_sequences,maxlen= max_len)

In [39]:
# train_matrix

array([[  0,   0,   0, ..., 229,   9,  23],
       [  0,   0,   0, ..., 121, 195, 182],
       [  0,   0,   0, ...,   2,  20,  65],
       ...,
       [  0,   0,   0, ...,   1, 825,  37],
       [  0,   0,   0, ..., 594, 645, 558],
       [  0,   0,   0, ...,  16,   1,   9]])

In [42]:
# Transforming to matrix
trainmatrix = tok.texts_to_matrix(x_train)
testmatrix = tok.texts_to_matrix(x_test)

In [16]:
trainmatrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [17]:
# encoder = LabelEncoder()
# encoder.fit(y_train)
# trainlabels = encoder.transform(y_train)
# testlabels = encoder.transform(y_test)

In [18]:
# trainlabels

array([2, 4, 3, ..., 4, 3, 4], dtype=int64)

In [223]:
num_classes = np.max(y_train) + 1
num_classes

6

In [224]:
trainlabels = to_categorical(y_train, num_classes)
testlabels = to_categorical(y_test, num_classes)

In [225]:
trainlabels

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [226]:
model= Sequential()
model.add(Dense(512, input_shape=(max_len,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [227]:
# Build and compile the model
# model = RNN()
model.compile(loss='binary_crossentropy',optimizer= 'adam',metrics=['accuracy'])    
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 512)               77312     
_________________________________________________________________
activation_23 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 3078      
_________________________________________________________________
activation_24 (Activation)   (None, 6)                 0         
Total params: 80,390
Trainable params: 80,390
Non-trainable params: 0
_________________________________________________________________


In [228]:
# Train the model
model.fit(train_matrix,trainlabels,validation_split = 0.2, 
           batch_size=32, epochs=5, 
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)]
         )

Train on 2077 samples, validate on 520 samples
Epoch 1/5
2077/2077 [==============================] - 1s 644us/step - loss: 3.4083 - accuracy: 0.7488 - val_loss: 3.2604 - val_accuracy: 0.7824
Epoch 2/5
2077/2077 [==============================] - 0s 125us/step - loss: 3.3780 - accuracy: 0.7675 - val_loss: 3.2243 - val_accuracy: 0.7625
Epoch 3/5
2077/2077 [==============================] - 0s 125us/step - loss: 3.3847 - accuracy: 0.7476 - val_loss: 3.3459 - val_accuracy: 0.7740


In [229]:
sequences = tok.texts_to_sequences(x_test)
test_matrix = sequence.pad_sequences(sequences,maxlen= max_len)

In [230]:
# Evaluate results for testing data
accr = model.evaluate(test_matrix,testlabels)

866/866 [==============================] - 0s 66us/step


In [231]:
# Print evaluation results
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
# Without lemmatization, with stop words, removed upper: 
#Test set
#  Loss: 3.447
#  Accuracy: 0.748

# With lemmatization, without stop words, removed upper: 
#Test set
#  Loss: 3.234
#  Accuracy: 0.772

# With spelling correction, without lemmatization, without stop words, removed upper 
# Test set
#  Loss: 3.303
#  Accuracy: 0.775

# With spelling correction, with lemmatization, without stop words, removed upper
#Test set
#  Loss: 3.301
#  Accuracy: 0.775

# With spelling correction, with lemmatization, without stop words, removed upper + remove non english words
#Test set
#  Loss: 3.348
#  Accuracy: 0.767

# With spelling correction, without lemmatization, without stop words, removed upper, with numbers
#Test set
#  Loss: 3.393
#  Accuracy: 0.764

# With spelling correction, without lemmatization, without stop words, removed upper, without numbers
#Test set
#  Loss: 3.401
#  Accuracy: 0.774

Test set
  Loss: 3.409
  Accuracy: 0.766


In [166]:
# newtest_sequences = tok.texts_to_sequences(feature_test)
# newtest_matrix = sequence.pad_sequences(newtest_sequences,maxlen=max_len)

newpred = model.predict(test_matrix) # for new data

In [167]:
newpred

array([[2.0290217e-30, 4.1300427e-06, 9.9998832e-01, 6.3614151e-21,
        1.0501322e-22, 7.6074621e-06],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.2960996e-25,
        2.1376202e-15, 1.0000000e+00],
       [0.0000000e+00, 8.2715907e-08, 7.4642692e-31, 7.3210509e-21,
        0.0000000e+00, 9.9999988e-01],
       ...,
       [1.0962534e-18, 4.8424402e-16, 1.4785289e-09, 1.1660159e-08,
        1.4437374e-19, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 3.7465810e-13, 0.0000000e+00,
        5.9582412e-14, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 9.9282912e-28, 0.0000000e+00,
        2.3225207e-27, 0.0000000e+00]], dtype=float32)

In [168]:
# Get the label with highest probability
predresults = []

for i in newpred:
    x = np.max(i) 
    for j in range(len(i)): 
        if i[j] == x:
            predresults.append(j)

In [169]:
predresults

[2,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 2,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 1,
 2,
 5,
 2,
 5,
 2,
 1,
 5,
 2,
 2,
 5,
 5,
 2,
 5,
 2,
 2,
 5,
 2,
 2,
 5,
 1,
 5,
 5,
 5,
 3,
 1,
 5,
 2,
 4,
 5,
 3,
 3,
 5,
 5,
 5,
 2,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 2,
 1,
 2,
 5,
 2,
 2,
 5,
 2,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 1,
 5,
 2,
 5,
 5,
 2,
 5,
 5,
 5,
 2,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 2,
 2,
 5,
 5,
 5,
 5,
 2,
 5,
 1,
 2,
 5,
 2,
 4,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 4,
 5,
 2,
 1,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 1,
 1,
 5,
 5,
 1,
 2,
 2,
 5,
 2,
 5,
 2,
 5,
 5,
 1,
 5,
 2,
 5,
 5,
 4,
 5,
 2,
 2,
 2,
 2,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 1,
 2,
 1,
 5,
 5,
 2,
 2,
 2,
 5,
 5,
 2,
 5,
 5,
 2,
 2,
 2,
 2,
 5,
 2,
 5,
 5,
 5,
 2,
 5,
 1,
 4,
 5,
 5,
 2,
 5,
 2,
 3,
 2,
 1,
 2,
 5,
 2,
 5,
 5,
 5,
 2,
 5,
 2,
 5,
 1,
 1,
 5,
 2,
 5,
 5,
 1,
 5,
 5,
 1,
 2,
 5,
 1,
 5,
 5,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 5,
 2,
 3,
 5,
 5,
 5,
 1,
 5,
 4,
 5,


In [174]:
confusion_matrix = pd.crosstab(df['Label'], df['Outcome'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted   1   2  3   4    5
Actual                       
1           9  33  0   5   75
2          31  83  9  14  141
3          18  57  4   5   83
4          13  37  2   2   52
5          10  50  3   1  108


In [170]:
y_test

2211    4
431     1
262     1
1256    2
1119    2
       ..
265     1
1865    3
1299    2
1786    3
1213    2
Name: Label, Length: 845, dtype: int64

In [173]:
df = pd.DataFrame(x_test)
df['Outcome'] = predresults
df['Label'] = y_test
df

,Review_clean,Outcome,Label
2211,great atmosphere and the esquites were exquisite,2,4
431,cheap okay quality meat,5,1
262,the food is quite cheaply priced although not ...,5,1
1256,this place honestly the food is insultingly bad,2,2
1119,the pumpkin ball is hard to bite,5,2
...,...,...,...
265,even if the coffee and food wasn t that great ...,2,1
1865,this is one of my favorite musical groups and ...,2,3
1299,and apple kouign amann was a little too sweet ...,5,2
1786,i m still getting used to the fact that they d...,1,3


In [109]:
y_train

270     3
517     5
1609    4
2004    5
846     2
       ..
2763    3
905     3
1096    5
235     4
1061    5
Name: Stars, Length: 2122, dtype: int64

In [110]:
df = pd.DataFrame(x_train)
df['Label'] = predresults
df

,Review_clean,Label
270,i may be demanding on this but i did not fores...,4
517,can t wait to come back for more,5
1609,it a spot i like to take people who are visiti...,5
2004,not expensive either,5
846,at 9 i found the price steep initially but the...,5
...,...,...
2763,service wa great,4
905,on top of that the only beverage you get are a...,4
1096,quite the contrast from the previous dish,5
235,creamy custard covered with the sugar glaze an...,4


## Getting TFIDF scores

In [31]:
# To get frequent words using TFIDF

word_scores = {}

def summarize(review_id): # works for each row

    # choose a random review that is at least 400 characters - keep looping until review has > 400 words 
#     review_length = 0
#     while review_length < 400:
#         review_id = np.random.randint(0, data.shape[0])
    review_text = reviews_by_restaurant.loc[review_id, 'compiled']
#         review_length = len(review_text)
    
    # create a list of all unique words in the review (minus stop words) using CountVectorizer
    vect = CountVectorizer(stop_words='english')
    vect.fit([review_text])
    unique_words = vect.get_feature_names()

    # create a dictionary of words and their TF-IDF scores
#     word_scores = {}
    for word in unique_words:
        word_scores[word] = dtm[review_id, features.index(word)]
    
    # print words with the top 5 TF-IDF scores
    print('TOP TF-IDF SCORING WORDS:')
    top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[0:5]
    for word, score in top_scores:
        print(word)
    
    # print 5 random words (for comparison)
    print('\n' + 'RANDOM WORDS:')
    random_words = np.random.choice(list(word_scores.keys()), size=5, replace=False)
    for word in random_words:
        print(word)
    
    # print the review
    print('\n' + review_text)

In [32]:
for i in range(len(reviews_by_restaurant)): 
    summarize(i)

TOP TF-IDF SCORING WORDS:
laksa
bowl
noodles
cockles
small

RANDOM WORDS:
ingredientsworth
non
colors
eggs
shrimp

food phenomenal laksa awesome cheap yet heavygot hawker center immediately saw sungei road laksa line formed small wait got bowl delicious laksathe soup dish delicious nice coconut flavor noodles also smooth protein went well dishoverall good bowl laksa price pay would eat im singaporelaksa popular southeast asia spicy noodle soup rice noodle coconut milk curry soup im big fan curry try good definitely worth try handle heat even though dont like curry still enjoyed sungei road seems best also tried laksa colony buffet ritz carlton thought quality pretty similar laksa fraction price 3 small bowllaska thing trying still thing love noodles soup prefer simpler clear broth although im fan soup see popular flavor profile curry coconut chili â bowls small good size youre americas think small chicken bowl size bowl soboro noodles cut utensils available spoons fits perfectly every 

TOP TF-IDF SCORING WORDS:
laksa
crab
cockles
soup
bowls

RANDOM WORDS:
jalan
precut
dedicated
pick
explain

first seafood meal favorite äyã­ family â loves taking love eat perfect â family always makes reservations since several uswe always get 2 large chilli crabs eveyone â absolutely delicious spicy â cant spice refrain trying missing â also got fried rice clams vegetables coconut drinks kids beers adults 1 large white pepper crabservice phenomenal always excellent food â another one favoritesintroduced place friend lives singapore less known location commercial jumbo seafood others get local feel service great explain items make sure know large crab know made big dent final tab ordered white pepper crab chili crab first time white pepper crab enjoyed chili crab chili crab mala taste chili also liked dipping mantou white pepper crab also got clams veggies tasty done well offer gloves utensils get coconut drink crabs came around 15kg check 300 one crab plenty 3 people even 4 wentdesi

TOP TF-IDF SCORING WORDS:
soup
pork
teh
kut
bak

RANDOM WORDS:
dad
ambiance
hahano
discovered
albeit

thanks yelp directing restaurant first meal singapore everi wasnt convinced place warranted reviews saw couple patsy white ribs clearish broth could anything delicious looked boiled seasonings nope nope nope stand corrected admit wrong meat unlike ribs ever broth extra bonus something unassuming simple good pretend even know socks completely knocked soup make grown woman cry yes yes ambiance service simple price low product amazing fly back singapore ribs soup mightbest food ive singapore service fast food came almost immediately bak kut teh die traditional singaporean dish consists braised pork ribs tea broth broth rich rich garlic flavor meat practically falls bone large portion around 10 singaporean dollars 722 us dollars plenty seating get cramped since theres many customers however miss food ton would sure come back ever singaporebest bak kut teh porkrib soup singapore â many good

TOP TF-IDF SCORING WORDS:
crab
chili
good
food
sauce

RANDOM WORDS:
kids
google
freshly
itemized
accommodating

three things buds red dot said must try â tian tian hainanese chicken rice bah kuh teh â singapore chili crab â crossed first two various hawker centers around town â chili crab many recommend come â arrived via metro around 1300 â restaurant half capacity â lunch rush crowd gone reservation needed â yesthe food appetizer shrimp pumpkin purã©e â really good â shrimp fried rice another great dish â enough 2 peoplemain singapore chili crab â excellent â flavorful â spice level right â enough kick enjoy â generous portion good last biteenvironmentservice great â waiter attentive â also let know ordered enough 2 people â glad stopped order two dishes â love bibs large hand washing stations â dont luxury meal worth price â glad listened buds pointed wayunfortunately rating isnt food never got chance try anything never felt unwanted disrespected life took solo trip singapore really

TOP TF-IDF SCORING WORDS:
pepper
kut
bak
seafood
jumbo

RANDOM WORDS:
chaser
lot
weekend
normally
starsafter

dining real treat â enjoyed venue much â located inside large contemporary museum â often get walk thru closed museum quiet intimate bustling oh charming restaurant like â step back time history lining walls framed photos culture embodied gorgeous chandeliers artworks â classy night dress bit feel part ambiance â service friendliest weve anywhere â host made excellent suggestions enjoyed every dish â reservations normally requisite though got lucky arriving without party finished early graciously accommodated usthink miss hit miss situation work holiday party weeks ago learned colleague next dinner used live singapore told eat order buah keluak ayam would change life made reservations dinner last nightwhen mentioned people going lot hype someone said chef national treasure concierge hotel said able get restaurant local surprised able secure reservation anticipation really high 

...

TOP TF-IDF SCORING WORDS:
murtabak
chor
bak
song
fa

RANDOM WORDS:
eveningsgreat
means
refuse
emerald
famoussaw

naked finnwell known seafood dinner tried many times always good especially cold vermicelli prawns tiny squids lunch first time limited menu seafood local foodlobster roll good lobster overcookednaked finn bar called nekkid nearbykeep seeing hairdresser posting yummy food finally last saturday night good foodnice ambience5 us shared one set one set b enough foodliked kang kong loved crispy squid bee hoon scallop carpacciobarramundi fish goodprawn sauce good great although waiter seemed think greatrecently traveled singapore first time heard leading trip clean country excellent food rang true experience found naked fin yelp looking restaurants around love pair food good glass wine images provided yelp page immediately restaurants dont offer drink list small popup shops outdoors humidity bit change socal weather haha thats naked fin stood video quick little blurb owner naked f

TOP TF-IDF SCORING WORDS:
chicken
murtabak
rice
chor
bak

RANDOM WORDS:
moreno
kahali
refillablemall
flow
lucked

thanks yelp helped find favorite hainanese chicken rice place world far got quarter chicken rice set 1580 confuses since thats actually quite expensive hear people say 27 reviews go something wrong price aside first bite rice looked like plain white rice btw chickeny rice heaven chicken perfectly juicy soup good im sold almost always trouble finishing rice places gobbled every last bit meal tried tian tian ah tai still rate highest terms overall taste set obviously 16 twice expensive tian tians chicken juicy sauce rice wasnt comparabledelish inexpensive good fast food 2 good hainesese chicken rice would recommend anyone wants get world hcr trying locals everyone tastes version lighter compared others market oh warned spicy chili sauce super spicy complements well ginger sauce chicken oh yumsimply delicious fast servicechicken rice full flavor08 sgd rice essential go chicken

TOP TF-IDF SCORING WORDS:
murtabak
bak
song
fa
ï¼

RANDOM WORDS:
deliciousâ
feels
annoyingfirst
unimpressive
salads

really best cantonese food around super authentic mouthwateringly delicious incredibly reasonable dont know could possibly ask coming years often crave im back la sure singapore full delicious chinese food food hub however every dish cooked perfection thats sets hong kong street apart cantonese restaurants opinionthere many yummy dishes go phases favorites current favorite pa ting yu steamed silver catfish ask mine black bean sauce firstly fish steamed perfection moist meltinyourmouth goodness secondly sauce point tend steer away black bean sauces salty usually however one anything tell make sauces least thats tastes like something bottle bursting flavor yet salty another dish like sum lor hor fun sliced fish hor fun arent familiar stir fried flat rice noodles bean sprouts sliced fish gravy soupy dry noodles either isnt however black singaporean hor fun might used like o

TOP TF-IDF SCORING WORDS:
murtabak
bak
song
fa
ï¼

RANDOM WORDS:
palatekeep
pursuit
paneer
order6th
hahaknow

much much much better jewel branch nearby official launch last friday course drop see curiosity expecting long queue surprise wasnt around 9ish overenthusiastic staff cheering screaming giving high fives walking thought buy iphone split second lol queueing order breeze 4 counters least 30 staff kitchen churning burgers fries concretes milkshakes efficiently impressive wait manageable around 20 mins order also staff dining floor assisting customers queries passing cutleries interacting kids basically great ambassador brand dining area big spacious enough lots seating booths tall tables plants onto burgers tried various burgers shake shack decided try chicknshack delicious chicken breast slowcooked buttermilk marinate fried perfection served lettuce pickles buttermilk mayo good stuff also tried concretes term used frozen custard blended mixins shack attack regular cantgowrong cho

KeyboardInterrupt: 

In [26]:
word_scores

{'10pm': 1,
 '15': 1,
 'ambiance': 1,
 'arrived': 1,
 'ate': 1,
 'bank': 1,
 'bar': 1,
 'bartenders': 2,
 'basil': 1,
 'bc': 1,
 'blinded': 1,
 'bourbon': 1,
 'break': 1,
 'brought': 1,
 'catch': 1,
 'chill': 1,
 'cocktail': 1,
 'delicious': 1,
 'didnt': 1,
 'disappointed': 1,
 'dont': 1,
 'evening': 1,
 'excellent': 1,
 'experience': 1,
 'fashioned': 1,
 'finely': 1,
 'friday': 2,
 'got': 4,
 'great': 1,
 'hunger': 1,
 'informative': 1,
 'kitchen': 1,
 'lovely': 1,
 'menu': 1,
 'minced': 1,
 'minutes': 1,
 'new': 2,
 'night': 1,
 'oil': 1,
 'old': 1,
 'olive': 1,
 'olives': 1,
 'open': 1,
 'owner': 1,
 'passionate': 1,
 'phenomenal': 1,
 'piggypower': 1,
 'pizza': 2,
 'place': 2,
 'prosciutto': 1,
 'relaxing': 1,
 'remember': 1,
 'right': 1,
 'served': 1,
 'shared': 1,
 'special': 1,
 'spicy': 1,
 'start': 1,
 'theyd': 1,
 'thing': 1,
 'told': 1,
 'warm': 1,
 'wines': 1,
 'work': 1,
 'yummy': 1}